In [1]:
# pip install marker-pdf

In [ ]:
import os
import re
import torch
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_community.llms import HuggingFacePipeline
from langchain_core.documents import Document
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.config.parser import ConfigParser
import pypdfium2 as pdfium


os.environ["HF_HUB_OFFLINE"] = "1"
os.environ["TRANSFORMERS_OFFLINE"] = "1"


In [ ]:
file_path = "SPRI_AI_Brief_2023년12월호_F.pdf"
num_chunk_size = 1000
num_chunk_overlap = 50

# step 1 : load document using marker-pdf
config_parser = ConfigParser({
    "output_format": "markdown",
    "paginate_output": True  # 페이지 구분 활성화
})
converter = PdfConverter(
    config=config_parser.generate_config_dict(),
    artifact_dict=create_model_dict(),
)

rendered = converter(file_path)

# 페이지별로 분리하여 LangChain Document로 변환
pages = re.split(r'\{(\d+)\}-+\n', rendered.markdown)
docs = []
for i in range(1, len(pages), 2):
    page_num = int(pages[i])
    content = pages[i + 1] if i + 1 < len(pages) else ""
    if content.strip():  # 빈 페이지 제외
        docs.append(Document(
            page_content=content.strip(),
            metadata={"source": file_path, "page": page_num}
        ))

# step 2 : split document
text_splitter = RecursiveCharacterTextSplitter(chunk_size=num_chunk_size, chunk_overlap=num_chunk_overlap)
split_documents = text_splitter.split_documents(docs)

# step 3 : Embedding
# embeddings = OpenAIEmbeddings()
hf_embeddings = HuggingFaceEmbeddings(model_name = "BAAI/bge-m3", model_kwargs={"device": "cuda"}, encode_kwargs={"normalize_embeddings": True},)

# step 4 : vector DB
try:
    vectorstore = FAISS.load_local(
        folder_path="faiss_db",
        index_name="faiss_index",
        embeddings=hf_embeddings,
        allow_dangerous_deserialization=True,
    )
except:
    vectorstore = FAISS.from_documents(documents=split_documents, embedding=hf_embeddings)
    vectorstore.save_local("faiss_db", "faiss_index")

# vectorstore.add_documents(new_split_documents)
# vectorstroe.save_local("faiss_db", "faiss_index")

/home/user/source/python312/.venv_python312/lib/python3.12/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  warnings.warn(
Recognizing tables: 100%|██████████| 1/1 [00:00<00:00,  1.51it/s]
Detecting bboxes: 0it [00:00, ?it/s]


In [3]:
# step 5 : Retriever Search
retriever = vectorstore.as_retriever()

# step 6 : generate prompt
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Answer in Korean.

#Question:
{question}

#Context:
{context}

#Answer:"""
)

# step 7 : LLM
# llm = ChatOpenAI(model="gpt-5-nano", temperature=0)
llm = ChatOllama(model="gpt-oss:20b", temperature=0, base_url="http://localhost:11434")

# step 8: chain
def format_docs(docs):
    return "\n\n".join(
        f"[page {d.metadata.get('page', 0) + 1}] {d.page_content}" for d in docs
    )
chain = (
    {"context": retriever | RunnableLambda(format_docs), "question":RunnablePassthrough()}
    | prompt | llm | StrOutputParser()
)


In [4]:
result = chain.invoke("삼성이 만든 생성AI 의 이름은 무엇인가요?")
print(result)

삼성이 만든 생성AI의 이름은 **“삼성 가우스”**입니다.


In [5]:
result = chain.invoke("AI 행동 강령에 대해 알려줘")
print(result)

AI 행동 강령은 첨단 AI 시스템을 개발·배포하는 기업이 자발적으로 따라야 할 국제적 규범입니다. 최근 G7(미국·일본·독일·영국·프랑스·이탈리아·캐나다)과 미국 정부가 발표한 주요 내용은 다음과 같습니다.

| 항목 | 핵심 내용 |
|------|-----------|
| **위험 식별·완화** | AI 수명주기 전반에 걸쳐 위험을 평가하고, 출시·배포 후에도 취약점·오용 사고를 모니터링·완화합니다. |
| **투명성·책임성** | 시스템의 성능·한계, 적절·부적절 사용 영역을 공개하고, 책임 있는 사용을 보장합니다. |
| **정보공유·협력** | 산업계, 정부, 시민사회, 학계 등 이해관계자 간에 위험 정보를 공유하고, 사고 발생 시 신고 체계를 마련합니다. |
| **보안 통제** | 물리·사이버·내부자 위협에 대한 강력한 보안 조치를 적용합니다. |
| **콘텐츠 인증·출처 확인** | 생성 AI가 만든 콘텐츠에 대해 출처와 진위 여부를 확인할 수 있는 기준을 마련합니다. |
| **개인정보 보호** | 위험 기반 접근법에 따라 개인정보 보호 정책을 수립하고, 데이터 사용을 제한합니다. |
| **국제 협력** | G7 국가 간 협의를 통해 행동 강령을 정기적으로 개정·업데이트합니다. |

### 미국 바이든 행정명령(2023년 10월 30일)
- **안전·보안 기준**: 고성능 AI(10^26 FLOPS 초과 등) 개발 기업은 안전 테스트 결과와 주요 정보를 미국 정부와 공유하도록 요구합니다.
- **개인정보 보호**: AI가 개인 정보를 처리할 때 강화된 보호 조치를 적용합니다.
- **형평성·시민권**: 주택, 법률, 보건 등 분야에서 AI 사용으로 인한 차별·편견을 방지하기 위한 지침을 마련합니다.
- **소비자 보호·근로자 지원**: 의료·교육·근로자 보호를 위한 원칙과 모범 사례를 제시합니다.
- **혁신·경쟁 촉진**: AI 기술 발전과 국제 협력을 통해 미국의 경쟁력을 강화합니다.

### 요약
- **G7 AI 행동 강령**은 첨단